In [1]:
import os
%pwd
os.chdir("../")

In [2]:
import os
from box.exceptions import BoxValueError
import yaml

import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any
import base64
from pathlib import Path

In [3]:
def create_directories(path_to_dir:list,verbose=True):
    for path in path_to_dir:
        os.makedirs(path,exist_ok=True)
       # if verbose:
           # logger.info(f"created dierctor ast {path}")


In [4]:
def read_yaml(path_to_yaml:Path)-> ConfigBox:
    try:
        with open(path_to_yaml) as yaml_file:
            content=yaml.safe_load(yaml_file)
           # logger.info(f"yaml file {path_to_yaml} load sucessfuly")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise(e)

In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class PredictionConfig:
    root_dir: Path
    source_name: str

In [15]:
class ConfigurationManger:
    def __init__(self,config_file_path,params_file_path):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_prediction_config(self) -> PredictionConfig:
        config=self.config.prediction

        create_directories([config.root_dir])
        prediction_config=PredictionConfig(


            root_dir=config.root_dir,


            
            source_name= config.source_name,
           
           
                
        )
      
        return prediction_config

In [12]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer


In [24]:
import torch

In [25]:
class Prediction:
     def __init__(self, config:PredictionConfig):
        self.config=config 
        self.model = AutoModelForSequenceClassification.from_pretrained(self.config.source_name,num_labels=5)
        self.tokenizer= AutoTokenizer.from_pretrained(self.config.source_name)
        

     def prediction(self,text):
         
         inputs = self.tokenizer(text, return_tensors="pt")
         
         with torch.no_grad():
            outputs = self.model(**inputs)
    
         # Get the predicted label (logits)
         logits = outputs.logits
         
         # Convert logits to probabilities
         probabilities = torch.softmax(logits, dim=1)
         
         # Get the label with the highest probability
         predicted_label = torch.argmax(probabilities, dim=1).item()
         
         return predicted_label, probabilities



         
         return predicted_label , probabilities


         



In [30]:
try:
    config=ConfigurationManger(config_file_path=Path("config.yaml"),params_file_path=Path("params.yaml"))
    prediction_config=config.get_prediction_config()
    pred=Prediction(config=prediction_config)
    label=pred.prediction("i love this product so much")
except Exception as e :
    raise e

In [31]:
label

(4, tensor([[0.0023, 0.0019, 0.0085, 0.0930, 0.8943]]))